# Create ELT aperture files

Need:
- aperture
- indexed aperture containing segment positions
- Lyot stop (empty for HARMONI)
- apodizer

In [ ]:
import os
from astropy.io import fits
import hcipy
import matplotlib.pyplot as plt
import numpy as np

## Telescope aperture

In [ ]:
pupil_grid = hcipy.make_pupil_grid(1034, diameter=40)

# Plain aperture
aper_gen = hcipy.make_elt_aperture(normalized=False)
aper = hcipy.evaluate_supersampled(aper_gen, pupil_grid, 4)

In [ ]:
# Indexed aperture
aper_ind_gen, seg_gen = hcipy.make_elt_aperture(normalized=False, segment_transmissions=np.arange(798)+1,
                                                return_segments=True, with_spiders=False)
aper_ind = hcipy.evaluate_supersampled(aper_ind_gen, pupil_grid, 4)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
hcipy.imshow_field(aper, cmap='Greys_r', origin='lower')
plt.colorbar()
plt.subplot(1,2,2)
hcipy.imshow_field(aper_ind, origin='lower')
plt.colorbar()

In [ ]:
type(seg_gen[0])

In [ ]:
this = seg_gen.points

In [ ]:
hdu_indexed = fits.PrimaryHDU()
hdr_indexed = hdu_indexed.header.copy()

In [ ]:
# seg_xys = seg_gen.points
# for segment in range(seg_xys.shape[0]):
#     print('seg' + str(segment + 1) + '_x', seg_gen.x[segment], 'x-position segment ' + str(segment + 1))
#     print('seg' + str(segment + 1) + '_y', seg_gen.y[segment], 'y-position segment ' + str(segment + 1))
#     hdr_indexed.set('seg' + str(segment + 1) + '_x', seg_gen.x[segment], 'x-position segment ' + str(segment + 1))
#     hdr_indexed.set('seg' + str(segment + 1) + '_y', seg_gen.y[segment], 'y-position segment ' + str(segment + 1))

In [ ]:
for i in range(len(seg_gen)):
    seg_xys = seg_gen[i].keywords['p']
    hdr_indexed.set('seg'+str(i+1)+ '_x',seg_xys[0],'x-position segment '+str(i+1))
    hdr_indexed.set('seg'+str(i+1)+ '_y',seg_xys[1],'y-position segment '+str(i+1))

In [ ]:
fits.writeto('ELT_aperture.fits', aper.shaped, overwrite=True)
fits.writeto('ELT_aperture_indexed.fits', aper_ind.shaped, hdr_indexed, overwrite=True)

## Empty LS and apodizer as stand-ins

In [ ]:
lyot_stop = np.ones_like(aper)
fits.writeto('lyot_stop.fits', lyot_stop.shaped, overwrite=True)

## Inspect apodizer files

In [ ]:
aper_fname = '/Users/ilaginja/Documents/MakidonLabWork/PASTIS/ELT_harmoni/ELT_1132_1024.fits'
spc_fname = '/Users/ilaginja/Documents/MakidonLabWork/PASTIS/ELT_harmoni/HSP2.fits'

aper = fits.getdata(aper_fname)
spc = fits.getdata(spc_fname)

print(aper.shape)
print(spc.shape)

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(aper, cmap='Greys_r', origin='lower')
plt.subplot(1, 2, 2)
plt.imshow(spc, cmap='Greys_r', origin='lower')

In [ ]:
# Replot after cutting down to 1024 x 1024 pixels to see what that gives on the edges
hbox = int((aper.shape[0] - 1034) / 2)
print(hbox)

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.imshow(aper[hbox:-hbox, hbox:-hbox], cmap='Greys_r', origin='lower')
plt.subplot(1, 2, 2)
plt.imshow(spc[hbox:-hbox, hbox:-hbox], cmap='Greys_r', origin='lower')

In [ ]:
#aper2 = aper[hbox:-hbox, hbox:-hbox]
spc2 = spc[hbox:-hbox, hbox:-hbox]
plt.figure(figsize=(15, 15))
plt.imshow(spc2, cmap='Greys_r', origin='lower')

In [ ]:
fits.writeto('spc_test.fits', spc2, overwrite=True)